<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Neural_ODEs_(Ordinary_Differential_Equations).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torchdiffeq

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchdiffeq import odeint

class ODEFunc(nn.Module):
    def __init__(self):
        super(ODEFunc, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(2, 50),
            nn.Tanh(),
            nn.Linear(50, 2),
        )

    def forward(self, t, y):
        return self.net(y)

class NeuralODE(nn.Module):
    def __init__(self, ode_func):
        super(NeuralODE, self).__init__()
        self.ode_func = ode_func

    def forward(self, y0, t):
        return odeint(self.ode_func, y0, t)

# Check device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize ODE function and Neural ODE
ode_func = ODEFunc().to(device)
neural_ode = NeuralODE(ode_func).to(device)
optimizer = optim.Adam(neural_ode.parameters(), lr=1e-3)

# Dummy data
y0 = torch.tensor([[0.5, 0.5]], dtype=torch.float32).to(device)
t = torch.linspace(0, 25, 100).to(device)

# Training loop
for epoch in range(100):
    optimizer.zero_grad()
    y_pred = neural_ode(y0, t)
    loss = ((y_pred - y0) ** 2).sum()
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")